# Text generation with LSTM

## Prepare data

In [1]:
import tensorflow as tf
import numpy as np

path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [2]:
# Length of extracted character sequences
maxlen = 60

# Sample a new sequence every `step` characters
step = 5

# Extracted sequences
sentences = []

# This holds the targets (follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# One-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 120167
Unique characters: 57
Vectorization...


## Build network

In [3]:
from keras import layers

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(tf.keras.layers.Dense(len(chars), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [ ]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Training model and sampling from it

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # Generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Instructions for updating:
Use tf.cast instead.
120167/120167 [==============================] - 111s 923us/sample - loss: 2.1241
--- Generating with seed: "t prizes more highly
than the other. each surrenders to the "
------ temperature: 0.2
t prizes more highly
than the other. each surrenders to the such the such and philosophy has a seals wish the some a self-selight and as the sumple the haster as is a sours and a soul a sours and a self--and and a self--what which the such and which the age as the has the such as the such as the such and a sectine and a self--and the such and the seals the such and a such as the such the himself as the such as the such as the seals the such and the suct is
------ temperature: 0.5
s the such as the such as the seals the such and the suct is in elles nother the such and nother the such and the sure the ever a to which which a secting and and the ellation the every and contire of a seaple acconce, and the same howe.


5. a gamant that a secte

soulity of suche, in ufogasion a deducfuled 2; they say love his reame of with and lent, to a there is
------ temperature: 1.2
d 2; they say love his reame of with and lent, to a there is higher betwomting-keriesc of couls.--must it. their pre;entness, bachpent
crawage, everifial-tess the of umatter:
if an
ether, phnrymitding, must matter diars-honekins precereing is o": what wishes whatherscles" in them-ear, for-ascymond seuotical accomituate ndenksneys himar
appoits. to, the oragene--ptecicusion thewe as it shilesages," the artits.
to gentincage suparilation
redienc," an emploue
epoch 5
120167/120167 [==============================] - 111s 922us/sample - loss: 1.4849
--- Generating with seed: "ss and stupidity in which
it has finally presented itself to"
------ temperature: 0.2
ss and stupidity in which
it has finally presented itself to the good and soul and consting the strive still one of the strive still still still in the string the such as the consting and soul its light of the

ion of the other who casse and sears the self-derust and the excerm sew vaunge can myself, but he carisfulness of man that we sumprond us, sear of
eadince to hel there is no writh is the ambit to do this similal
so mad. i, becume him beef the feeven mare because minden an eternt of an
acts. the sole, are also the phewoprng to
divine
posse fact, for his veryonable
them lomeful imporagine itsal melogical education of mmy
understands?--as very be very liatter
------ temperature: 1.2
gical education of mmy
understands?--as very be very liatter man
tend distrusts, a them--otess, as tooverlyg-the. vay moragened, mebity, theoist infadly the virtueliactin was a whole unfirst lifed, hon eltion and
one
sank in nexureous the reaming man
using, meanculon would
then mad"--thereby perfessing ridity and nuwatic, as they will" untaiceying fraited . lough rerttent a batters to belong. to what it exical actetiond irrees for
sensitive his
land course
epoch 9
120167/120167 [==============================]